In [8]:
import pandas as pd
import sqlalchemy
import re
from sqlalchemy import text

In [3]:
!pip install pymysql

In [ ]:
# ---- DB connection ----
USER = "..."
PASS = "..."
HOST = "..."
PORT = ...
DB   = "youtube_analytics"

engine = sqlalchemy.create_engine(
    f"mysql+pymysql://{USER}:{PASS}@{HOST}:{PORT}/{DB}?charset=utf8mb4"
)

In [ ]:
# ---- What to export----
TABLES = [
    "channel_data",
    "video_data",
]
VIEWS = [
    "vw_corr_pearson",
    "vw_monthly_kpis",
    "vw_top15_by_engagement",
    "vw_top15_by_views",
    "vw_top_categories",
    "vw_video_metrics",
    "vw_video_type_summary",
]

OBJECTS = [("table", t) for t in TABLES] + [("view", v) for v in VIEWS]

In [ ]:
# ---- Helpers ----
def sanitize_sheet_name(name: str) -> str:
    name = re.sub(r'[:\\/?*\[\]]', '-', name)
    return name[:31] or "sheet"

def fetch_all(conn, obj_name: str) -> pd.DataFrame:
    return pd.read_sql(text(f"SELECT * FROM `{obj_name}`"), conn)

In [ ]:
# ---- Export to a single .xlsx ----
outfile = "youtube_analytics_export.xlsx"

with engine.begin() as conn, pd.ExcelWriter(outfile, engine="openpyxl") as writer:
    written = []
    for kind, name in OBJECTS:
        try:
            df = fetch_all(conn, name)
            sheet = sanitize_sheet_name(name)
            orig_sheet = sheet
            suffix = 2
            while sheet in written:
                sheet = sanitize_sheet_name(f"{orig_sheet}_{suffix}")
                suffix += 1
            df.to_excel(writer, index=False, sheet_name=sheet)
            written.append(sheet)
            print(f"✓ Wrote {kind}: {name}  →  sheet: {sheet}  (rows: {len(df)})")
        except Exception as e:
            print(f"⚠ Skipped {kind}: {name}  (error: {e})")

print(f"\nDone. Excel saved to: {outfile}")

✓ Wrote table: channel_data  →  sheet: channel_data  (rows: 1)
✓ Wrote table: video_data  →  sheet: video_data  (rows: 601)
✓ Wrote view: vw_corr_pearson  →  sheet: vw_corr_pearson  (rows: 5)
✓ Wrote view: vw_monthly_kpis  →  sheet: vw_monthly_kpis  (rows: 130)
✓ Wrote view: vw_top15_by_engagement  →  sheet: vw_top15_by_engagement  (rows: 15)
✓ Wrote view: vw_top15_by_views  →  sheet: vw_top15_by_views  (rows: 15)
✓ Wrote view: vw_top_categories  →  sheet: vw_top_categories  (rows: 5)
✓ Wrote view: vw_video_metrics  →  sheet: vw_video_metrics  (rows: 601)
✓ Wrote view: vw_video_type_summary  →  sheet: vw_video_type_summary  (rows: 3)

Done. Excel saved to: youtube_analytics_export.xlsx
